In [2]:
import random as r

from engine import Value
from modules import *
from ops import *
from gpt import *

In [3]:
batch_size = 2
vocab_len = 10
model_dim = 8
max_seq_len = 5
seq_len = 3
num_heads = 2
head_dim = 4
mlp_mult = 4

In [ ]:
class CrossEntropyLoss(Module):
    def __init__(self, pad_token):
        self.pad_token = pad_token

    def __call__(self, logits, targets):
        '''
        inputs: 
        logits - list of lists of lists of shape (batch_size, seq_len, vocab_len) full of Value objects
        targets - list of lists of shape (batch_size, seq_len) full of integers representing token indices

        output: a single Value object representing loss of the model
        '''
        # idk how i went straight to this without doing Log first

In [8]:
x = Value(2.718)
print(x)
y = x.log()
print(y)
y.grad = 1.0
y._backward()
print(y)
print(x)

Value(data=2.718, grad=0.000)
Value(data=1.000, grad=0.000)
Value(data=1.000, grad=1.000)
Value(data=2.718, grad=0.368)


In [13]:
def log(vec):
    '''
    takes the natural log of all elements in the vector
    '''
    assert isinstance(vec, list), "vec should be a list of Value objects"
    assert all(isinstance(x, Value) for x in vec), "All elements in vec must be Value objects"
    return [x.log() for x in vec]

In [17]:
x = [Value(r.uniform(-1,1)) for _ in range(model_dim)]
print(x)
y = log(softmax(x))
print(y)

x = [[[Value(r.uniform(-1,1)) for _ in range(model_dim)]
      for _ in range(seq_len)]
     for _ in range(batch_size)]
pretty_print_tensor(x)
y = vector_wise_apply(log, vector_wise_apply(softmax, x))
pretty_print_tensor(y)

[Value(data=-0.330, grad=0.000), Value(data=-0.055, grad=0.000), Value(data=0.393, grad=0.000), Value(data=-0.823, grad=0.000), Value(data=0.685, grad=0.000), Value(data=0.408, grad=0.000), Value(data=0.684, grad=0.000), Value(data=0.064, grad=0.000)]
[Value(data=-2.645, grad=0.000), Value(data=-2.369, grad=0.000), Value(data=-1.921, grad=0.000), Value(data=-3.138, grad=0.000), Value(data=-1.630, grad=0.000), Value(data=-1.906, grad=0.000), Value(data=-1.631, grad=0.000), Value(data=-2.251, grad=0.000)]
[
  [
    [Value(data=0.944, grad=0.000), Value(data=0.555, grad=0.000), Value(data=0.215, grad=0.000), Value(data=0.810, grad=0.000), Value(data=-0.920, grad=0.000), Value(data=-0.456, grad=0.000), Value(data=0.556, grad=0.000), Value(data=0.064, grad=0.000)]
    [Value(data=-0.479, grad=0.000), Value(data=-0.745, grad=0.000), Value(data=0.126, grad=0.000), Value(data=0.850, grad=0.000), Value(data=0.338, grad=0.000), Value(data=-0.197, grad=0.000), Value(data=-0.261, grad=0.000), Valu

In [ ]:
# TODO: make Embedding module also do unembedding w/ shared weights? 
# wouldn't be exactly faithful to pytorch implementation but i'd like to use gradient accumulation & save parameters

In [14]:
x = [[[Value(r.uniform(-1,1)) for _ in range(model_dim)]
      for _ in range(seq_len)]
     for _ in range(batch_size)]
print(get_shape(x))
layer = ResidualLayer(model_dim, num_heads, head_dim, max_seq_len, mlp_mult)
y = layer(x)
print(get_shape(y))

[2, 3, 8]
[2, 3, 8]
